## 1. Upload raw data files into S3

In [ ]:
!pip install pyspark

In [ ]:
!pip install boto3

In [ ]:
import pandas as pd

import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, to_timestamp, to_date

In [ ]:
config = configparser.ConfigParser()
config.read('configs/global.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

KEY = config.get('AWS', 'AWS_ACCESS_KEY_ID')
SECRET = config.get('AWS','AWS_SECRET_ACCESS_KEY')

In [ ]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

### Manage AWS instances

In [ ]:
import boto3

#### EC2

In [28]:
ec2 = boto3.client('ec2', 
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET, 
                   region_name='us-west-2')

In [ ]:
s3 = boto3.client( 's3', 
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET, 
                   region_name='us-west-2')

In [ ]:
# find udacity bucket
emr_bucket = [entry['Name'] for entry in s3.list_buckets()['Buckets'] if 'udacity' in entry['Name']  ]
emr_bucket

In [ ]:
s3.upload_file( 'etl.py', emr_bucket[0], 'etl.py' )

In [ ]:
s3.put_object( 
    Body='https://zenodo.org/record/4568860/files/covid19_2020_02_15.json.gz' , 
    Bucket=emr_bucket[0] ,
    Key = 'covid19_2020_02_15.json.gz'
)

In [ ]:
s3.upload_file('https://zenodo.org/record/4568860/files/covid19_2020_02_15.json.gz', emr_bucket[0] , 'test_upload.gz')